In [208]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import sys
%run ../data/states_districts.py

In [222]:
# Fucntion which recieves state and district in the form e.g. TN07 as arg to scrape page
def scrape_candidate_data(state_district):
    endpoint = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state_district}&spec=N'
    res = requests.get(endpoint)
    soup = BeautifulSoup(res.text, 'html.parser')
    pattern_candidate = re.compile(r"(.+?) \((R|D|I)\)( •Incumbent•Winner)?(\(([\d.]+)% of vote\))?")
    # Your existing code...
    pandas_data = []

    # Extract candidate data
    for element in soup.find_all('h2'):
        text = element.get_text(strip=True)
        match = pattern_candidate.match(text)
        if match:
            name, party, incumbent_winner, _, vote_percentage = match.groups()
            incumbent = incumbent_winner is not None
            winner = incumbent  # Assumes if they are incumbent they are also the winner
            pandas_data.append({
                "Name": name,
                "Party": party,
                "Incumbent": incumbent,
                "Winner": winner,
                "Vote Percentage": vote_percentage,
                "Raised": None,
                "Spent": None
            })

    financial_data = []

    # Loop through all the tables with class 'Members--table'
    for table in soup.find_all('table', class_='Members--table'):
        # Initialize a dictionary to store the raised and spent values for this table
        table_data = {}
        
        # Loop through all the tr elements in the table
        for row in table.find_all('tr'):
            # Get the text content of all td elements in the row
            cols = [col.get_text() for col in row.find_all('td')]
            
            # Check if the first column is 'Raised' or 'Spent', and if so, store the data
            if cols[0] == 'Raised:':
                table_data['Raised'] = cols[1]
            elif cols[0] == 'Spent:':
                table_data['Spent'] = cols[1]
        
        # Append the data dictionary to the financial_data list
        financial_data.append(table_data)

    # Check if lengths of pandas_data and financial_data match
    if len(pandas_data) == len(financial_data):
        # Update pandas_data with financial data
        for p_data, f_data in zip(pandas_data, financial_data):
            p_data.update(f_data)
    else:
        print("Mismatch in the number of entries between candidate data and financial data")

    return pandas_data

In [221]:
data = scrape_candidate_data("CA19")
df = pd.DataFrame(data)
df

,Name,Party,Incumbent,Winner,Vote Percentage,Raised,Spent
0,Zoe Lofgren,D,True,True,71.7,"$1,755,555","$1,731,700"
1,Justin Aguilera,R,False,False,28.3,"$9,112","$11,542"
